In [1]:
import importlib
import local_utils; importlib.reload(local_utils)
from local_utils import *

Using TensorFlow backend.


Import time: 13.83356785774231
Import time: 0.00010466575622558594


In [2]:
global_start = time()

In [3]:
train_df = pd.read_csv(DATA_DIR + '/train_v2.csv')
test_df = pd.read_csv(DATA_DIR + '/sample_submission_v2.csv')

label_map, inv_label_map, Y = process_labels(train_df)
print("Shape of Y {}".format(Y.shape))

train_folds, valid_folds = stratified_kfold_sampling(Y, n_splits=10, random_state=1000)
print("Shape of train folds {}".format(train_folds.shape))
print("Shape of valid folds {}".format(valid_folds.shape))

image_size = 128

Shape of Y (40479, 17)
Shape of train folds (10,)
Shape of valid folds (10,)


In [4]:
model_name = "deep_model"

fold_inx = 1
batch_size = 80
all_steps = np.ceil(len(Y)/batch_size)
train_steps = np.ceil(len(train_folds[fold_inx])/batch_size)
valid_steps = np.ceil(len(valid_folds[fold_inx])/batch_size)
test_steps = np.ceil(len(test_df)/batch_size)

In [5]:
# cache_images(DATA_DIR+"/train_images_"+str(image_size)+".dat", train_df['image_name'].values, DATA_DIR + '/train-jpg', True, image_size, image_size, dtype=np.float32)
# cache_images(DATA_DIR+"/test_images_"+str(image_size)+".dat", test_df['image_name'].values, DATA_DIR + '/test-jpg', True, image_size, image_size, dtype=np.float32)

In [6]:
train_array = load_array(DATA_DIR+'/train_images_'+str(image_size)+'.dat')

In [7]:
x_input = Input(shape=(image_size, image_size, 3))

x = BatchNormalization()(x_input)
x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
xm = MaxPooling2D()(x)
xa = AveragePooling2D()(x)

xm = BatchNormalization()(xm)
xm = Conv2D(64, (3, 3), padding='same', activation='relu')(xm)
xm = Conv2D(64, (3, 3), activation='relu')(xm)
xm = MaxPooling2D()(xm)

xm = BatchNormalization()(xm)
xm = Conv2D(128, (3, 3), padding='same', activation='relu')(xm)
xm = Conv2D(128, (3, 3), activation='relu')(xm)
xm = MaxPooling2D()(xm)

xm = BatchNormalization()(xm)
xm = Conv2D(256, (3, 3), padding='same', activation='relu')(xm)
xm = Conv2D(256, (3, 3), activation='relu')(xm)
xm = GlobalMaxPooling2D()(xm)

xa = BatchNormalization()(xa)
xa = Conv2D(64, (3, 3), padding='same', activation='relu')(xa)
xa = Conv2D(64, (3, 3), activation='relu')(xa)
xa = AveragePooling2D()(xa)

xa = BatchNormalization()(xa)
xa = Conv2D(128, (3, 3), padding='same', activation='relu')(xa)
xa = Conv2D(128, (3, 3), activation='relu')(xa)
xa = AveragePooling2D()(xa)

xa = BatchNormalization()(xa)
xa = Conv2D(256, (3, 3), padding='same', activation='relu')(xa)
xa = Conv2D(256, (3, 3), activation='relu')(xa)
xa = GlobalAveragePooling2D()(xa)

x = concatenate([xm, xa])

x = Dense(512, activation='relu')(x)

x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)

x_output = Dense(17, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=x_input, outputs=x_output)
model.summary()

In [8]:
def getImageDataGenerator():
    return ImageDataGenerator(
                rotation_range=90,
                horizontal_flip=True,
                vertical_flip=True,)

all_gen = getImageDataGenerator()
train_gen = getImageDataGenerator()
valid_gen = getImageDataGenerator()
test_gen = getImageDataGenerator()

In [9]:
f2_history = F2History(train_array[valid_folds[fold_inx]], Y[valid_folds[fold_inx]], 
                       valid_gen, valid_steps, batch_size)

model_checkpoint = ModelCheckpoint(
                DATA_DIR+'/models/'+model_name+'_'+str(fold_inx)+'.h5', 
                monitor='val_fn_loss', 
                verbose=1, 
                save_best_only=True, 
                save_weights_only=False, 
                mode='min',
                period=1)

In [ ]:
# del model
# model = load_model(DATA_DIR+'/models/'+model_name+'_'+str(fold_inx)+'.h5', compile=False)

In [ ]:
# opt=optimizers.Adam(lr=1e-3, decay=0.5e-5); epochs=20 # 0.33 30
# opt=optimizers.Adam(lr=1e-4, decay=0.1e-5); epochs=10 # 0.66 15
# opt=optimizers.Adam(lr=1e-5, decay=0.1e-6); epochs=5
# opt=optimizers.Adam(lr=0.5e-5, decay=0.1e-6); epochs=5 #?
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fn_loss, f2, 'accuracy'])

In [ ]:
model.fit_generator(
    generator=train_gen.flow(train_array[train_folds[fold_inx]], Y[train_folds[fold_inx]], batch_size=batch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=valid_gen.flow(train_array[valid_folds[fold_inx]], Y[valid_folds[fold_inx]], batch_size=batch_size, shuffle=False),
    validation_steps=valid_steps,
    callbacks=[model_checkpoint, f2_history])

In [ ]:
print(f2_history.f2_02_scores)
plt.plot(f2_history.f2_02_scores)
plt.show()

In [ ]:
print(f2_history.f2_05_scores)
plt.plot(f2_history.f2_05_scores)
plt.show()

In [ ]:
stop

In [101]:
del model
model = load_model(DATA_DIR+'/models/'+model_name+'_'+str(fold_inx)+'.h5', compile=False)

opt=optimizers.Adam(lr=1e-5, decay=0.1e-6); epochs=5;
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fn_loss, f2, 'accuracy'])

In [102]:
# model.fit_generator(
#     generator=valid_gen.flow(train_array[valid_folds[fold_inx]], Y[valid_folds[fold_inx]], batch_size=batch_size),
#     steps_per_epoch=valid_steps,
#     epochs=epochs)

Epoch 1/5
51/51 [==============================] - 28s - loss: 0.0900 - fn_loss: 0.1114 - f2: 0.8975 - acc: 0.9657    
Epoch 2/5
51/51 [==============================] - 26s - loss: 0.0917 - fn_loss: 0.1137 - f2: 0.8937 - acc: 0.9648    
Epoch 3/5
51/51 [==============================] - 27s - loss: 0.0909 - fn_loss: 0.1119 - f2: 0.8984 - acc: 0.9658    
Epoch 4/5
51/51 [==============================] - 27s - loss: 0.0913 - fn_loss: 0.1129 - f2: 0.8958 - acc: 0.9650    
Epoch 5/5
51/51 [==============================] - 27s - loss: 0.0910 - fn_loss: 0.1130 - f2: 0.8942 - acc: 0.9646    


In [103]:
# model.evaluate(train_array[train_folds[fold_inx]], Y[train_folds[fold_inx]], batch_size=batch_size)
# model.save(DATA_DIR+'/models/'+model_name+'_'+str(fold_inx)+'.h5')

36430/36430 [==============================] - 79s    


[0.078270561357071985,
 0.097470962605304912,
 0.90946767452448951,
 0.96901553416036168]

In [106]:
model.evaluate_generator(
    valid_gen.flow(train_array[valid_folds[fold_inx]], Y[valid_folds[fold_inx]], batch_size=batch_size, shuffle=False),
    steps=valid_steps)

[0.087895764791029243,
 0.10899029899950703,
 0.90044955881883126,
 0.96645506030690609]

In [38]:
# model.evaluate_generator(
#     all_gen.flow(train_array, Y, batch_size=batch_size, shuffle=False),
#     steps=all_steps)

In [107]:
p_X_no_tta, p_X_tta_list = predictTTA(model, train_array, all_gen, all_steps, batch_size, 10)

506/506 [==============================] - 86s    


In [108]:
p_X_tta = sum(p_X_tta_list)

In [109]:
c = 11.0
p_X = (c*p_X_no_tta+p_X_tta)/(c+10.0)

In [112]:
start = time()
thres = find_best_thresholds(Y[valid_folds[fold_inx]], p_X[valid_folds[fold_inx]])
print("F2 Valid Score: {}".format(f2_score(Y[valid_folds[fold_inx]], p_X[valid_folds[fold_inx]], thres)))
print("F2 All Score: {}".format(f2_score(Y, p_X, thres)))

save_array(DATA_DIR+'/models/'+model_name+'_thres_'+str(fold_inx)+'.dat', thres)
print(time() - start)

/opt/conda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0: Score 0.9273439251311457 with 0.297
4: Score 0.927050590753084 with 0.203
1: Score 0.9270483290812743 with 0.209
7: Score 0.9270454691607924 with 0.219
2: Score 0.9271454898947575 with 0.169
5: Score 0.9274536317458528 with 0.28650000000000003
6: Score 0.927818510755448 with 0.096
3: Score 0.9271710534536511 with 0.1695
8: Score 0.9271329448524258 with 0.196
11: Score 0.9270790462605422 with 0.2135
14: Score 0.9270931147542137 with 0.11950000000000001
9: Score 0.9272809471159886 with 0.2335
12: Score 0.9273494192811049 with 0.248
13: Score 0.9273578145478347 with 0.2715
10: Score 0.9272197151562164 with 0.2315
15: Score 0.9270642106737617 with 0.21
16: Score 0.9272157781065562 with 0.2115
F2 Valid Score: 0.9302978596910565
F2 All Score: 0.9323065168506792
29.770496129989624


In [113]:
# thres = load_array(DATA_DIR+'/models/'+model_name+'_thres_'+str(fold_inx)+'.dat')
print(f2_score(Y[valid_folds[fold_inx]], p_X[valid_folds[fold_inx]], thres))
print(f2_score(Y, p_X, thres))

0.930297859691
0.932306516851


In [ ]:
# max_f2 = 0.0
# best_c = 0
# for c in range(1, 15):
#     print(c)
#     p_X = (c*p_X_no_tta+p_X_tta)/(10.0 + c)
#     thres = find_best_thresholds(Y[valid_folds[fold_inx]], p_X[valid_folds[fold_inx]])
#     current_f2 = f2_score(Y[valid_folds[fold_inx]], p_X[valid_folds[fold_inx]], thres)
    
#     print("F2 Valid Score: {}".format(current_f2))
#     print("F2 All Score: {}".format(f2_score(Y, p_X, thres)))
    
#     if current_f2>max_f2:
#         max_f2 = current_f2
#         best_c = c

# print(max_f2)
# print(best_c)

In [115]:
del train_array
del p_X
del p_X_no_tta
del p_X_tta
del p_X_tta_list
del model
del thres

start = time()
model = load_model(DATA_DIR+'/models/'+model_name+'_'+str(fold_inx)+'.h5', compile=False)
thres = load_array(DATA_DIR+'/models/'+model_name+'_thres_'+str(fold_inx)+'.dat')

test_array = load_array(DATA_DIR+'/test_images_'+str(image_size)+'.dat')
print(time() - start)

20.89811897277832


In [116]:
p_X_test_no_tta, p_X_test_tta_list = predictTTA(model, test_array, test_gen, test_steps, batch_size, 10)
p_X_test_tta = sum(p_X_test_tta_list)

765/765 [==============================] - 129s   


In [126]:
c = 20.0
p_X_test = (c*p_X_test_no_tta+p_X_test_tta)/(c+10.0)

In [127]:
start = time()
for inx in np.arange(len(test_df)):
    test_df['tags'][inx] = ' '.join([inv_label_map[i] for i, b in enumerate(p_X_test[inx] >= thres) if b])
print(time() - start)

test_df.head()

14.340683698654175


,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,partly_cloudy primary
3,test_3,clear cultivation primary
4,test_4,cloudy partly_cloudy primary


In [128]:
save_array(DATA_DIR+'/results/'+model_name+'_test.dat', p_X_test)
test_df.to_csv(DATA_DIR+'/results/'+model_name+'_submission.csv', index=False)
FileLink('data/results/'+model_name+'_submission.csv')

/src/DL/planet_understanding_the_amazon_from_space/data/results/deep_model_submission.csv

In [124]:
print((time() - global_start)/60/60)

2.618086404403051
